In [1]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [2]:
# get data
data = pd.read_csv('Level_2_ZyMoC1_EsCoC2_PsPuC3_1188.csv') 

# split data
X=data.iloc[:,1:51]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

          Xc1.A      Xc1.R     Xc1.N  ...  Xc2.lambda28  Xc2.lambda29  Xc2.lambda30
0     14.767060  10.829177  7.383530  ...      0.025877      0.025830      0.026292
1     11.728905   8.138424  3.351116  ...      0.025056      0.025025      0.026281
2      8.319776   2.773259  1.663955  ...      0.022828      0.023152      0.024468
3     10.255686   3.778411  2.428978  ...      0.023815      0.022707      0.024686
4      6.745884   2.698354  0.809506  ...      0.024331      0.024572      0.023173
...         ...        ...       ...  ...           ...           ...           ...
1183   3.434023   2.113245  1.849089  ...      0.024686      0.022914      0.025301
1184   8.597566   6.513308  3.647452  ...      0.025071      0.024687      0.022506
1185   8.597410   1.719482  3.152384  ...      0.024239      0.024058      0.021686
1186   5.471856   0.521129  1.563388  ...      0.022357      0.026263      0.022232
1187   5.471856   0.521129  1.563388  ...      0.022357      0.026263      0

In [3]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [4]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.20,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(50, 3 - 1) = 2 components.
  ChangedBehaviorWarni

In [5]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(3 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2, 50)             10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 128)            91648     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 151,651
Trainable params: 151,651
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
44/44 [==============================] - 38s 94ms/step - loss: 0.6776 - accuracy: 0.4598 - val_loss: 0.5712 - val_accuracy: 0.5130
Epoch 2/1000
44/44 [==============================] - 1s 17ms/step - loss: 0.5930 - accuracy: 0.4598 - val_loss: 0.5369 - val_accuracy: 0.5942
Epoch 3/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.5611 - accuracy: 0.5714 - val_loss: 0.4919 - val_accuracy: 0.7159
Epoch 4/1000
44/44 [==============================] - 1s 17ms/step - loss: 0.5364 - accuracy: 0.5878 - val_loss: 0.4403 - val_accuracy: 0.6986
Epoch 5/1000
44/44 [==============================] - 1s 17ms/step - loss: 0.4910 - accuracy: 0.6146 - val_loss: 0.4251 - val_accuracy: 0.7072
Epoch 6/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.5054 - accuracy: 0.6205 - val_loss: 0.4172 - val_accuracy: 0.7130
Epoch 7/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.5019 - accuracy: 0.6227 - val_loss: 0.4207 - val_accuracy: 0.718

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.6339869281045751
test-acc = 0.6521739130434783
train-CFR =               precision    recall  f1-score   support

           0       0.73      0.54      0.62       453
           1       0.58      0.67      0.62       469
           2       0.63      0.70      0.66       455

    accuracy                           0.63      1377
   macro avg       0.65      0.63      0.63      1377
weighted avg       0.65      0.63      0.63      1377

test-CFR =               precision    recall  f1-score   support

           0       0.76      0.57      0.65       121
           1       0.56      0.70      0.62       105
           2       0.68      0.69      0.68       119

    accuracy                           0.65       345
   macro avg       0.66      0.65      0.65       345
weighted avg       0.67      0.65      0.65       345

